In [1]:
import pandas as pd

# 读取三个结果文件和原始数据集
predicted_emotions = pd.read_csv('predicted_emotions.csv')
predicted_emotions_painting_and_utterance = pd.read_csv('predicted_emotions_painting_and_utterance.csv')
predicted_emotions_only_on_utterance = pd.read_csv('predicted_emotions_only_on_utterance.csv')
original_dataset = pd.read_csv('1k_batch.csv')

# 合并数据集以便对比
merged_results = original_dataset[['painting', 'emotion']].merge(
    predicted_emotions[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_predicted')
).merge(
    predicted_emotions_painting_and_utterance[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_painting_and_utterance')
).merge(
    predicted_emotions_only_on_utterance[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_only_on_utterance')
)

# 计算准确率
def calculate_accuracy(row):
    return {
        'original': row['emotion'],
        'only_image': row['predicted_emotion'],
        'image_and_utterance': row['predicted_emotion_painting_and_utterance'],
        'only_utterance': row['predicted_emotion_only_on_utterance'],
        'correct_only_image': row['emotion'] in row['predicted_emotion'],
        'correct_image_and_utterance': row['emotion'] in row['predicted_emotion_painting_and_utterance'],
        'correct_only_utterance': row['emotion'] in row['predicted_emotion_only_on_utterance']
    }

comparison_results = merged_results.apply(calculate_accuracy, axis=1, result_type='expand')

accuracy_only_image = comparison_results['correct_only_image'].mean()
accuracy_image_and_utterance = comparison_results['correct_image_and_utterance'].mean()
accuracy_only_utterance = comparison_results['correct_only_utterance'].mean()

# 输出准确率
print(f"Accuracy (Only Image): {accuracy_only_image:.2%}")
print(f"Accuracy (Image and Utterance): {accuracy_image_and_utterance:.2%}")
print(f"Accuracy (Only Utterance): {accuracy_only_utterance:.2%}")

# 保存对比结果到CSV文件
comparison_results.to_csv('comparison_results.csv', index=False)


Accuracy (Only Image): 36.90%
Accuracy (Image and Utterance): 10.00%
Accuracy (Only Utterance): 6.60%


In [2]:
import pandas as pd

# 读取三个结果文件和原始数据集
predicted_emotions = pd.read_csv('predicted_emotions.csv')
predicted_emotions_painting_and_utterance = pd.read_csv('predicted_emotions_painting_and_utterance.csv')
predicted_emotions_only_on_utterance = pd.read_csv('predicted_emotions_only_on_utterance.csv')
original_dataset = pd.read_csv('1k_batch.csv')

# 合并数据集以便对比
merged_results = original_dataset[['painting', 'emotion', 'utterance']].merge(
    predicted_emotions[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_predicted')
).merge(
    predicted_emotions_painting_and_utterance[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_painting_and_utterance')
).merge(
    predicted_emotions_only_on_utterance[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_only_on_utterance')
)

# 添加准确性标记
merged_results['correct_only_image'] = merged_results['emotion'] == merged_results['predicted_emotion']
merged_results['correct_image_and_utterance'] = merged_results['emotion'] == merged_results['predicted_emotion_painting_and_utterance']
merged_results['correct_only_utterance'] = merged_results['emotion'] == merged_results['predicted_emotion_only_on_utterance']

# 保存合并后的结果到CSV文件
merged_results.to_csv('merged_comparison_results.csv', index=False)

# 分析错误案例
incorrect_cases_only_image = merged_results[~merged_results['correct_only_image']]
incorrect_cases_image_and_utterance = merged_results[~merged_results['correct_image_and_utterance']]
incorrect_cases_only_utterance = merged_results[~merged_results['correct_only_utterance']]

# 输出一些错误案例以供分析
print("Incorrect cases (Only Image):")
print(incorrect_cases_only_image[['painting', 'emotion', 'utterance', 'predicted_emotion']].head())

print("\nIncorrect cases (Image and Utterance):")
print(incorrect_cases_image_and_utterance[['painting', 'emotion', 'utterance', 'predicted_emotion_painting_and_utterance']].head())

print("\nIncorrect cases (Only Utterance):")
print(incorrect_cases_only_utterance[['painting', 'emotion', 'utterance', 'predicted_emotion_only_on_utterance']].head())

# 分析情绪标签的分布
emotion_distribution = original_dataset['emotion'].value_counts()
print("\nEmotion distribution in the dataset:")
print(emotion_distribution)


Incorrect cases (Only Image):
                                            painting    emotion  \
0              agostino-carracci_venus-and-mars-1600  amusement   
1                    diego-velazquez_self-portrait-1  amusement   
2                        felicien-rops_tienne-soubre  amusement   
3      koloman-moser_animal-motif-for-a-picture-book  amusement   
4  martiros-saryan_woman-with-mask-s-i-dymshits-1913  amusement   

                                           utterance  \
0  All of the figures appear to have very importa...   
1  The mans collar looks like a college graduatio...   
2  The head is ridiculously large and the legs ar...   
3  The little green shape reminds me of a bird an...   
4  The facial expression indicates that woman is ...   

                                predicted_emotion  
0  This artwork conveys a feeling of contentment.  
1      This artwork conveys a feeling of disgust.  
2      This artwork conveys a feeling of disgust.  
3  This artwork convey